In [1]:
import numpy as np
import arcode
import codec
import struct
import os
import matplotlib.pyplot as plt
import matplotlib as mpl
import skimage
from PIL import Image

In [2]:
def show_data(x_in, pre_in):
    mean = np.mean(x_in)
    std = np.std(x_in)
    Max = np.max(x_in)
    Min = np.min(x_in)
    dtype = x_in.dtype
    
    print(pre_in + ' shape: ' + str(x_in.shape) + ' , mean = ' + str(mean) 
          + ' , std = ' + str(std) + ' , Max = ' + str(Max) + ' , Min = ' + str(Min) 
          + ' , type=' + str(dtype))

In [3]:
def oc_fname(i_in):
    rname = '/f00' + str(i_in) + '.png'
    if i_in >9: rname = '/f0' + str(i_in) + '.png'
    if i_in >99: rname = '/f' + str(i_in) + '.png'
        
    return rname

In [4]:
def read_COMPINT(prefC, cname):
    o4=np.zeros([1, nx, ny, ncomp])
    for k in range(ncomp):
        inname = prefC + str(k) + cname
        #print("  read_COMPINT: " + inname)
        tim=Image.open(inname)
        tima=np.array(tim)
        o4[0,:,:,k] = tima
                
    return o4

In [5]:
def ar_code(x_in, oname):
    img = x_in.flatten()
    olen = len(img)
    img = img.reshape(olen,1,1)
    
    bits = codec.bitstream(dat=img)
    bits.h = img.shape[0]
    bits.w = img.shape[1]
    bits.c = img.shape[2]
    entro_coder = codec.entropy_codec(Mode='Arith')
    entro_coder.encode(bits, oname)
    
    size1= os.path.getsize(oname + 'dat')
    size2= os.path.getsize(oname + 'arith')

    #olen, size1, size2 in unit of Byte (assuming input is 8bit int)
    return olen, size1, size2

In [6]:
def ar_decode(aname, dname):
    ar = arcode.ArithmeticCode(False)
    ar.decode_file(aname, dname)

    file_bin = open(dname,'rb')
    s = file_bin.read()
    H,W,C = struct.unpack('3i',s[:12])
    #print('H= ' + str(H) + ' ; W= ' + str(W) + ' ; C= ' + str(C))
    
    bytes=[]
    for i in range(H*W*C):
        bytes.append(struct.unpack('b',s[12+i:12+i+1]))
    file_bin.close()
    bin_data = np.array(bytes).reshape(H,W,C)
    bin_data = bin_data.flatten()
    
    return bin_data

In [33]:

pref = '/home/saberbud/Programs/xy_TF_sectioned_compression/Kodak_Bc_884/COMPINT'
nx=256
ny=384
ncomp=4


'''
pref = '/home/saberbud/Programs/Paper_RB_14bit_COMPINTs/CI16164/COMPINT'
nx=128
ny=192
ncomp=4
'''

'''
pref = '/home/saberbud/Programs/xy_TF_sectioned_compression/Kodak_Bc_32324/COMPINT'
nx=64
ny=96
ncomp=4
'''

print(pref)

/home/saberbud/Programs/xy_TF_sectioned_compression/Kodak_Bc_884/COMPINT


In [34]:
i_f = 7
t_C = read_COMPINT(pref, oc_fname(i_f))

show_data(t_C, 't_C')
for i in range(4):
    show_data(t_C[...,i], 't_C_'+str(i))

t_C shape: (1, 256, 384, 4) , mean = 128.48583221435547 , std = 37.1043373455402 , Max = 206.0 , Min = 55.0 , type=float64
t_C_0 shape: (1, 256, 384) , mean = 95.66770426432292 , std = 10.297372519899534 , Max = 131.0 , Min = 55.0 , type=float64
t_C_1 shape: (1, 256, 384) , mean = 157.74102783203125 , std = 9.528964188928082 , Max = 195.0 , Min = 129.0 , type=float64
t_C_2 shape: (1, 256, 384) , mean = 169.47064208984375 , std = 12.710277747212622 , Max = 206.0 , Min = 134.0 , type=float64
t_C_3 shape: (1, 256, 384) , mean = 91.06395467122395 , std = 11.64008795119767 , Max = 127.0 , Min = 56.0 , type=float64


In [35]:
idc = 3
data_t = t_C[0,:,:,idc]
data_t = data_t - 128.0
data_t = data_t.astype(np.int8)
show_data(data_t, 'data_t')

data_t shape: (256, 384) , mean = -36.93604532877604 , std = 11.64008795119767 , Max = -1 , Min = -72 , type=int8


In [36]:
oname = 'fAc'
olen, size1, size2 = ar_code(data_t, oname)

print('olen' + str(olen))
print('size1' + str(size1))
print('size2' + str(size2))

ratio1 = float(size1)/float(olen)
ratio2 = float(size2)/float(olen)

print('ratio1= ' + str(ratio1))
print('ratio2= ' + str(ratio2))

olen98304
size198316
size263942
ratio1= 1.00012207031
ratio2= 0.650451660156


In [37]:
aname = oname + 'arith'
dname = 'dAc.dat'
dcode = ar_decode(aname, dname)

dlen = len(dcode)
print('dlen= ' + str(dlen))

mse = np.mean(np.square(dcode-data_t.flatten()))
print('mse = ' + str(mse))

dlen= 98304
mse = 0.0


In [38]:
fs = 1; fe = 25
nf = fe-fs
size1s = np.zeros([nf,4])
size2s = np.zeros([nf,4])
olens = np.zeros([nf,4])
d_lens = np.ones([nf,4])
mses = np.zeros([nf,4])

for i in range(fs,fe):
    print(i)
    t_C = read_COMPINT(pref, oc_fname(i))
    show_data(t_C, '  t_C')
    
    for idc in range(4):
        data_t = t_C[0,:,:,idc]
        data_t = data_t - 128.0
        data_t = data_t.astype(np.int8)
        show_data(data_t, ' channel'+str(idc))
        olen, size1, size2 = ar_code(data_t, oname)

        print('   olen: ' + str(olen) + ' Bytes')
        print('   size1: ' + str(size1) + ' Bytes')
        print('   size2: ' + str(size2) + ' Bytes')

        ratio1 = float(size1)/float(olen)
        ratio2 = float(size2)/float(olen)

        print('ratio1= ' + str(ratio1))
        print('ratio2= ' + str(ratio2))
        
        dcode = ar_decode(aname, dname)
        dlen = len(dcode)
        print('dlen= ' + str(dlen))

        idx = i-fs
        
        if dlen == olen:
            d_lens[idx,idc] = 0
            mse = np.mean(np.square(dcode-data_t.flatten()))
            mses[idx,idc] = mse
            print('mse = ' + str(mse))
        if dlen != olen:
            d_lens[idx,idc] = 1
            mses[idx,idc] = 99.99
            print('!Length differ!')
        
        size1s[idx,idc] = size1
        size2s[idx,idc] = size2
        olens[idx,idc] = olen
        print('')
    
    print('\n')

1
  t_C shape: (1, 256, 384, 4) , mean = 128.52951049804688 , std = 37.647287032764794 , Max = 212.0 , Min = 52.0 , type=float64
 channel0 shape: (256, 384) , mean = -33.377410888671875 , std = 8.620902110123735 , Max = 4 , Min = -70 , type=int8
   olen: 98304 Bytes
   size1: 98316 Bytes
   size2: 59979 Bytes
ratio1= 1.00012207031
ratio2= 0.610137939453
dlen= 98304
mse = 0.0

 channel1 shape: (256, 384) , mean = 30.768941243489582 , std = 8.156294731805707 , Max = 69 , Min = -6 , type=int8
   olen: 98304 Bytes
   size1: 98316 Bytes
   size2: 58911 Bytes
ratio1= 1.00012207031
ratio2= 0.599273681641
dlen= 98304
mse = 0.0

 channel2 shape: (256, 384) , mean = 42.838714599609375 , std = 9.735153407786466 , Max = 84 , Min = 3 , type=int8
   olen: 98304 Bytes
   size1: 98316 Bytes
   size2: 61403 Bytes
ratio1= 1.00012207031
ratio2= 0.624623616536
dlen= 98304
mse = 0.0

 channel3 shape: (256, 384) , mean = -38.112202962239586 , std = 9.289850068965505 , Max = 0 , Min = -76 , type=int8
   olen

dlen= 98304
mse = 0.0

 channel1 shape: (256, 384) , mean = 31.328379313151043 , std = 13.383018713822864 , Max = 70 , Min = -9 , type=int8
   olen: 98304 Bytes
   size1: 98316 Bytes
   size2: 69428 Bytes
ratio1= 1.00012207031
ratio2= 0.706258138021
dlen= 98304
mse = 0.0

 channel2 shape: (256, 384) , mean = 43.734415690104164 , std = 16.86128642143105 , Max = 84 , Min = -1 , type=int8
   olen: 98304 Bytes
   size1: 98316 Bytes
   size2: 72925 Bytes
ratio1= 1.00012207031
ratio2= 0.741831461589
dlen= 98304
mse = 0.0

 channel3 shape: (256, 384) , mean = -38.90789794921875 , std = 15.940230984111404 , Max = 5 , Min = -78 , type=int8
   olen: 98304 Bytes
   size1: 98316 Bytes
   size2: 72053 Bytes
ratio1= 1.00012207031
ratio2= 0.73296101888
dlen= 98304
mse = 0.0



9
  t_C shape: (1, 256, 384, 4) , mean = 128.56114451090494 , std = 39.955330508348986 , Max = 206.0 , Min = 51.0 , type=float64
 channel0 shape: (256, 384) , mean = -35.108367919921875 , std = 9.923887671390407 , Max = 2 , Min

   olen: 98304 Bytes
   size1: 98316 Bytes
   size2: 73337 Bytes
ratio1= 1.00012207031
ratio2= 0.746022542318
dlen= 98304
mse = 0.0

 channel3 shape: (256, 384) , mean = -34.742828369140625 , std = 19.757522488027472 , Max = 5 , Min = -76 , type=int8
   olen: 98304 Bytes
   size1: 98316 Bytes
   size2: 71853 Bytes
ratio1= 1.00012207031
ratio2= 0.730926513672
dlen= 98304
mse = 0.0



16
  t_C shape: (1, 256, 384, 4) , mean = 128.43494669596353 , std = 33.1898820073337 , Max = 207.0 , Min = 57.0 , type=float64
 channel0 shape: (256, 384) , mean = -28.410400390625 , std = 10.739584346824225 , Max = -3 , Min = -70 , type=int8
   olen: 98304 Bytes
   size1: 98316 Bytes
   size2: 58339 Bytes
ratio1= 1.00012207031
ratio2= 0.593454996745
dlen= 98304
mse = 0.0

 channel1 shape: (256, 384) , mean = 26.098347981770832 , std = 9.900914525244643 , Max = 59 , Min = 4 , type=int8
   olen: 98304 Bytes
   size1: 98316 Bytes
   size2: 56703 Bytes
ratio1= 1.00012207031
ratio2= 0.576812744141
dlen= 98304


dlen= 98304
mse = 0.0



23
  t_C shape: (1, 256, 384, 4) , mean = 128.47948455810547 , std = 37.46034689064024 , Max = 208.0 , Min = 56.0 , type=float64
 channel0 shape: (256, 384) , mean = -31.591176350911457 , std = 13.196292605621881 , Max = 0 , Min = -71 , type=int8
   olen: 98304 Bytes
   size1: 98316 Bytes
   size2: 66633 Bytes
ratio1= 1.00012207031
ratio2= 0.677825927734
dlen= 98304
mse = 0.0

 channel1 shape: (256, 384) , mean = 29.054178873697918 , std = 12.141141734816593 , Max = 60 , Min = 2 , type=int8
   olen: 98304 Bytes
   size1: 98316 Bytes
   size2: 65186 Bytes
ratio1= 1.00012207031
ratio2= 0.663106282552
dlen= 98304
mse = 0.0

 channel2 shape: (256, 384) , mean = 40.548685709635414 , std = 16.721405366851275 , Max = 80 , Min = 5 , type=int8
   olen: 98304 Bytes
   size1: 98316 Bytes
   size2: 70625 Bytes
ratio1= 1.00012207031
ratio2= 0.718434651693
dlen= 98304
mse = 0.0

 channel3 shape: (256, 384) , mean = -36.09375 , std = 14.959033549433363 , Max = 0 , Min = -72 ,

In [39]:
for idx in range(nf):
    print(idx+1)
    
    ratio_1 = np.mean(size1s[idx,:])/np.mean(olens[idx,:])
    ratio_2 = np.mean(size2s[idx,:])/np.mean(olens[idx,:])

    #print('   size_orig= ' + str(size_orig))
    print('   ratio_1= ' + str(ratio_1))
    print('   ratio_2= ' + str(ratio_2))
    print(' ')

1
   ratio_1= 1.0001220703125
   ratio_2= 0.6139933268229166
 
2
   ratio_1= 1.0001220703125
   ratio_2= 0.5142644246419271
 
3
   ratio_1= 1.0001220703125
   ratio_2= 0.6359354654947916
 
4
   ratio_1= 1.0001220703125
   ratio_2= 0.61199951171875
 
5
   ratio_1= 1.0001220703125
   ratio_2= 0.7118403116861979
 
6
   ratio_1= 1.0001220703125
   ratio_2= 0.6441141764322916
 
7
   ratio_1= 1.0001220703125
   ratio_2= 0.6406428019205729
 
8
   ratio_1= 1.0001220703125
   ratio_2= 0.7231547037760416
 
9
   ratio_1= 1.0001220703125
   ratio_2= 0.6364059448242188
 
10
   ratio_1= 1.0001220703125
   ratio_2= 0.6595789591471354
 
11
   ratio_1= 1.0001220703125
   ratio_2= 0.6196085611979166
 
12
   ratio_1= 1.0001220703125
   ratio_2= 0.5543848673502604
 
13
   ratio_1= 1.0001220703125
   ratio_2= 0.6776657104492188
 
14
   ratio_1= 1.0001220703125
   ratio_2= 0.69580078125
 
15
   ratio_1= 1.0001220703125
   ratio_2= 0.7197748819986979
 
16
   ratio_1= 1.0001220703125
   ratio_2= 0.60121663411